# Efficient Coding Assistant with simpleaichat

_Updated using `gpt-3.5-turbo-0613`_

Many coders use ChatGPT for coding help, however the web interface can be slow and contain unnecessary discussion when you want code. With some system prompt engineering and simplechatapi streaming, you can cut down code time generation and costs significantly.

**DISCLAIMER: Your mileage may vary in terms of code quality and accuracy in practice, but this is a good, hackable starting point.**



In [1]:
!pip install -q simpleaichat

from simpleaichat import AIChat
from getpass import getpass

For the following cell, input your OpenAI API key when prompted. **It will not be saved to the notebook**.

In [2]:
api_key = getpass("OpenAI Key: ")

OpenAI Key: ··········


In [3]:
params = {"temperature": 0.0}  # for reproducibility
model = "gpt-3.5-turbo"  # in production, may want to use model="gpt-4" if have access

ai = AIChat(api_key=api_key, console=False, params=params, model=model)

Let's start with a simple `is_palindrome()` function in Python, and track how long it takes to run. The output of this should be similar to what is shown in the ChatGPT webapp.

In [4]:
%%time
response = ai("Write an is_palindrome() function in Python.")
print(response)
print("\n\n")  # separate time from generated text for readability

Sure! Here's an example of an is_palindrome() function in Python:

```python
def is_palindrome(word):
    # Convert the word to lowercase and remove any spaces
    word = word.lower().replace(" ", "")
    
    # Check if the word is equal to its reverse
    if word == word[::-1]:
        return True
    else:
        return False
```

You can use this function to check if a word is a palindrome. It will return True if the word is a palindrome and False otherwise.



CPU times: user 23.7 ms, sys: 1.32 ms, total: 25 ms
Wall time: 2.49 s


That's the typical implementation. However, there's a trick to cut the processing time in half, well known by technical hiring managers who want to trip up prospective candidates.

ChatGPT outputs the statistically most common implementation, but it's not necessairily the best. A second pass allows ChatGPT to refine its output.

In [5]:
%%time
response = ai("Make it more efficient.")
print(response)
print("\n\n")

Certainly! Here's an optimized version of the is_palindrome() function that uses two pointers to check if a word is a palindrome:

```python
def is_palindrome(word):
    # Convert the word to lowercase and remove any spaces
    word = word.lower().replace(" ", "")
    
    # Initialize two pointers, one at the start and one at the end of the word
    left = 0
    right = len(word) - 1
    
    # Iterate until the pointers meet in the middle
    while left < right:
        # If the characters at the pointers are not equal, the word is not a palindrome
        if word[left] != word[right]:
            return False
        
        # Move the pointers towards the middle
        left += 1
        right -= 1
    
    # If the loop completes without returning False, the word is a palindrome
    return True
```

This optimized version reduces the number of comparisons by using two pointers that start at opposite ends of the word and move towards the middle. It will return True if the word is 

In [6]:
ai.total_length

511

In all, it took ~6 seconds and utilized 511 tokens. But there's a lot of unnecessary natter in the output:

- The conversational preamble before the code
- Docstrings and code comments
- A long explanation of the code which may be redundant to the above

All this natter adds latency and cost.

The easiest technique to guide AI text generation is to use **prompt engineering**, specifically to give it a new system prompt to say precisely what you want. As of June 27th 2023, the default ChatGPT API responds very well to commands.

Now, for the new `system` prompt:

In [7]:
system_optimized = """Write a Python function based on the user input.

You must obey ALL the following rules:
- Only respond with the Python function.
- Never put in-line comments or docstrings in your code."""

ai_2 = AIChat(api_key=api_key, system=system_optimized, model=model, params=params)

In [8]:
%%time
response = ai_2("is_palindrome")
print(response)
print("\n\n")

def is_palindrome(word):
    return word == word[::-1]



CPU times: user 11.6 ms, sys: 4.01 ms, total: 15.6 ms
Wall time: 1.04 s


In [9]:
%%time
response = ai_2("Make it more efficient.")
print(response)
print("\n\n")

def is_palindrome(word):
    length = len(word)
    for i in range(length // 2):
        if word[i] != word[length - i - 1]:
            return False
    return True



CPU times: user 19.7 ms, sys: 1.73 ms, total: 21.5 ms
Wall time: 2.64 s


In [10]:
ai_2.total_length

190

~3 seconds total with 190 tokens used: that's 2x faster at 1/3 the cost!

## Create a Function

Now we can create a function to automate the two calls we did above for any arbitrary input.

For each call, we'll create an independent temporary session within simpleaichat and then clean it up. We'll also use a regex to strip unneded backticks.

In [11]:
from uuid import uuid4
import re

ai_func = AIChat(api_key=api_key, console=False)
def gen_code(query):
    id = uuid4()
    ai_func.new_session(api_key=api_key, id=id, system=system_optimized, params=params, model=model)
    _ = ai_func(query, id=id)
    response_optimized = ai_func("Make it more efficient.", id=id)

    ai_func.delete_session(id=id)
    return response_optimized

In [12]:
%%time
code = gen_code("is_palindrome")
print(code)
print("\n\n")

def is_palindrome(word):
    length = len(word)
    for i in range(length // 2):
        if word[i] != word[length - i - 1]:
            return False
    return True



CPU times: user 27.8 ms, sys: 1.94 ms, total: 29.8 ms
Wall time: 1.96 s


In [13]:
%%time
code = gen_code("reverse string")
print(code)
print("\n\n")

def reverse_string(string):
    return ''.join(reversed(string))



CPU times: user 16.4 ms, sys: 644 µs, total: 17 ms
Wall time: 1.42 s


In [14]:
%%time
code = gen_code("pretty print dict")
print(code)
print("\n\n")

def pretty_print_dict(dictionary):
    import json
    print(json.dumps(dictionary, indent=4, separators=(',', ': ')))



CPU times: user 17.2 ms, sys: 1.87 ms, total: 19.1 ms
Wall time: 1.31 s


In [15]:
%%time
code = gen_code("load and flip image horizontally")
print(code)
print("\n\n")

import cv2

def load_and_flip_image(image_path):
    image = cv2.imread(image_path)
    flipped_image = cv2.flip(image, 1)
    cv2.imwrite("flipped_image.jpg", flipped_image)
    return "flipped_image.jpg"



CPU times: user 21.1 ms, sys: 3.86 ms, total: 24.9 ms
Wall time: 2.18 s


In [16]:
%%time
code = gen_code("multiprocess hash")
print(code)
print("\n\n")

import hashlib
from multiprocessing import Pool, cpu_count

def multiprocess_hash(data):
    def hash_string(string):
        return hashlib.sha256(string.encode()).hexdigest()

    num_processes = cpu_count()
    pool = Pool(processes=num_processes)
    hashed_data = pool.map(hash_string, data)
    pool.close()
    pool.join()

    return hashed_data



CPU times: user 28.8 ms, sys: 3.82 ms, total: 32.6 ms
Wall time: 3.31 s


## Generating Optimized Code in a Single API Call w/ Structured Output Data

Sometimes you may not want to make two calls to OpenAI. One hack you can do is to define an expected structured output to tell it to sequentially generate the normal code output, then the optimized output.

This structure is essentially a different form of prompt engineering, but you can combine it with a system prompt if needed.

This will also further increase response speed, but may not necessairly result in fewer tokens used.

In [23]:
from pydantic import BaseModel, Field
import orjson

class write_python_function(BaseModel):
    """Writes a Python function based on the user input."""
    code: str = Field(description="Python code")
    efficient_code: str = Field(description="More efficient Python code than previously written")

In [24]:
ai_struct = AIChat(api_key=api_key, console=False, model=model, params=params, save_messages=False)

In [25]:
%%time
response_structured = ai_struct("is_palindrome", output_schema=write_python_function)

# orjson.dumps preserves field order from the ChatGPT API
print(orjson.dumps(response_structured, option=orjson.OPT_INDENT_2).decode())
print("\n\n")

{
  "code": "def is_palindrome(s):\n    return s == s[::-1]",
  "efficient_code": "def is_palindrome(s):\n    n = len(s)\n    for i in range(n // 2):\n        if s[i] != s[n - i - 1]:\n            return False\n    return True"
}



CPU times: user 131 ms, sys: 1.39 ms, total: 133 ms
Wall time: 1.67 s


As evident, the output is a `dict` so you'd just return the `efficient_code` field.

In [26]:
print(response_structured["efficient_code"])

def is_palindrome(s):
    n = len(s)
    for i in range(n // 2):
        if s[i] != s[n - i - 1]:
            return False
    return True


In [27]:
ai_struct.total_length

161

Token-wise it's about the same, but there's a significant speedup in generation for short queries such as these.

Trying the other examples:

In [28]:
%%time
response_structured = ai_struct("reverse string", output_schema=write_python_function)
print(response_structured["efficient_code"])
print("\n\n")

def reverse_string(s):
    return ''.join(reversed(s))



CPU times: user 24.6 ms, sys: 250 µs, total: 24.9 ms
Wall time: 1.37 s


In [29]:
%%time
response_structured = ai_struct("load and flip image horizontally", output_schema=write_python_function)
print(response_structured["efficient_code"])
print("\n\n")

from PIL import Image

def load_and_flip_image_horizontally(image_path):
    return Image.open(image_path).transpose(Image.FLIP_LEFT_RIGHT)



CPU times: user 15.4 ms, sys: 2.15 ms, total: 17.6 ms
Wall time: 1.79 s


## MIT License

Copyright (c) 2023 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
